In [90]:
import pandas as pd 
import matplotlib.pyplot as plt
import requests
import numpy as np
from math import floor
from termcolor import colored as cl 
import seaborn as sns
from datetime import datetime

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['figure.figsize'] = (20, 10)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [91]:
# data import 
df = pd.read_excel('Time Series Data.xlsx', index_col = 'Day')
df.head(10)


,Series 1,Series 2,Series 3,Series 4,Series 5,Series 6,Series 7,Series 8,Series 9,Series 10,Series 11,Series 12,Series 13,Series 14,Series 15,Series 16,Series 17,Series 18,Series 19,Series 20,Series 21,Series 22,Series 23,Series 24,Series 25,Series 26,Series 27,Series 28,Series 29
Day,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,100.172920,100.589993,103.823464,98.644063,99.750482,106.614321,101.767999,100.732626,99.952576,100.227093,98.790108,100.032421,96.316754,101.460335,98.573111,100.032584,99.004063,100.390444,100.318979,101.568951,99.730607,99.673906,99.409461,99.113705,100.012563,100.311565,99.887188,96.556886,97.922078
2,98.579911,98.767737,107.058757,97.936942,99.220660,110.602917,100.681776,100.383846,99.617067,100.246064,98.754979,99.418521,99.700262,102.108065,98.256322,99.558198,98.782991,101.258255,102.048828,106.193229,101.541426,99.977798,99.577298,99.783831,99.682467,100.366287,99.902872,95.608782,100.000000
3,98.397809,97.087379,107.352843,100.000000,99.251805,113.424191,102.651998,101.569981,98.659381,101.230321,98.509591,98.384407,97.980366,100.108347,97.886884,99.018405,98.838321,100.940089,101.410870,106.936416,100.938414,99.278429,99.055138,100.875486,99.637414,100.767880,100.216365,95.109780,100.000000
4,99.051233,99.716206,105.587978,99.703996,99.407530,113.715275,100.710685,101.081407,99.760755,101.040888,96.686487,99.063150,99.663613,100.325042,98.679303,98.887350,98.340352,100.535001,101.521286,107.018993,101.448655,99.979185,99.602163,100.162127,100.379047,101.058262,100.323414,90.469062,99.480519
5,98.177450,97.318895,106.764672,99.056674,97.786202,117.800899,99.121810,98.221499,100.335509,101.097803,98.649591,97.802929,99.532723,98.918881,99.471721,98.494664,99.962949,101.663108,101.239112,95.293146,102.740313,100.693818,100.167837,99.535236,100.172412,101.234786,100.756144,87.574850,100.000000
6,97.061884,95.108290,103.529379,97.288125,96.102227,115.466257,96.960985,97.593600,98.707513,101.059860,99.719483,97.156608,97.924084,95.564820,99.154932,99.623606,100.921834,101.156924,99.116673,92.568126,100.685078,100.513425,100.491080,98.341980,98.868923,100.285086,100.409110,91.067864,97.142857
7,98.429944,96.422704,102.941207,98.820469,97.567830,113.618247,97.683150,98.116771,100.383641,101.684506,99.982435,98.061457,100.897906,96.862634,101.478646,100.654317,101.438328,100.752072,97.718071,92.568126,101.544102,100.679942,101.199726,98.767834,99.977041,101.120045,100.914686,89.071856,98.701299
8,97.860684,95.108290,103.235293,98.231451,96.725842,114.397456,97.371171,98.709605,102.873767,101.930570,100.157565,100.032421,95.344241,95.348125,102.377255,100.752548,99.962949,100.766480,97.399092,95.540875,101.900913,100.596684,101.541617,98.573281,100.360420,101.365414,101.196055,87.974052,97.662338


In [92]:
#RSI function
def get_rsi(close, window):
    ret = close.diff()
    up = []
    down = []
    for i in range(len(ret)):
        if ret.iloc[i] < 0:
            up.append(0)
            down.append(ret.iloc[i])
        else:
            up.append(ret.iloc[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    up_ewm = up_series.ewm(com = window - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = window - 1, adjust = False).mean()
    rs = up_ewm/down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df

In [93]:
# To generate position and the respective buying/ selling price
def implement_rsi_strategy(data, rsi, lower, upper):
    buy_price = []
    sell_price = []
    rsi_signal = []
    signal = 0
    price = 0 
        
    for i in range(len(rsi)):
        if rsi.iloc[i-1] > lower and rsi.iloc[i] < lower:
            if signal != 1:
                buy_price.append(data.iloc[i])
                sell_price.append(np.nan)
                signal = 1
                rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
        elif rsi.iloc[i-1] < upper and rsi.iloc[i] > upper:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(data.iloc[i])
                signal = -1
                rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            rsi_signal.append(0)

        position = []

    for i in range(len(rsi_signal)):    
        if rsi_signal[i] > 1:
            position.append(0)
        else:
            position.append(1)

    sigPriceBuy = []
    sigPriceSell = []

    for i in range(len(data)):
        if rsi_signal[i] == 1:
            position[i] = 1
#             sigPriceBuy.append(data.iloc[i])
#             sigPriceSell.append(np.nan)
        elif rsi_signal[i] == -1:
            position[i] = 0
#             sigPriceBuy.append(np.nan)
#             sigPriceSell.append(data.iloc[i])
        else:
            position[i] = position[i-1]
#             sigPriceBuy.append(np.nan)
#             sigPriceSell.append(np.nan)

    return position, buy_price, sell_price
            
       

In [94]:
def PnL(data, position):
    gain = 0
    real_profit = 0
    series_ret = pd.DataFrame(np.diff(data)).rename(columns = {0:'returns'})
    for i in range(len(series_ret)):
        try:
            returns = series_ret['returns'][i]*position[i]
            gain += returns 
        except:
            pass
    diff = (data.iloc[-1] - data.iloc[0])
    real_profit = gain - diff
    return real_profit, diff 


In [95]:
# Testing series 28

df['rsi'] = get_rsi(df['Series 1'], 14)
df_temp = df.copy()
df_temp['rsi'] = df_temp['rsi'][4:]
df_temp = df_temp.dropna()

position_temp = implement_rsi_strategy(df_temp['Series 1'], df_temp['rsi'], 30, 70)

position_df = pd.Series(position_temp)
profit_temp = PnL(df_temp['Series 1'], position_df[0])
print(profit_temp[0])


-656.376629736182


In [97]:
# Testing all data (RSI reading can be modified, current: 30/70)


profit_df = pd.DataFrame(columns = df.columns)
profits = pd.DataFrame(columns = df.columns).transpose()

profit = []

for column in df:
    temp_df = df.copy() 
    temp_df['rsi'] = get_rsi(temp_df[column], 14)
    temp_df['rsi'] = temp_df['rsi'][4:]
    temp_df = temp_df.dropna()
    position_temp = implement_rsi_strategy(temp_df[column], temp_df['rsi'], 30, 70)
    position_df = pd.Series(position_temp)
    profit_temp = PnL(temp_df[column], position_df[0])
    profit.append(profit_temp[0])

    #Graph Plotting
#     ax1 = plt.subplot2grid((10,1), (0,0), rowspan = 4, colspan = 1)
#     ax2 = plt.subplot2grid((10,1), (5,0), rowspan = 4, colspan = 1)
#     ax1.plot(temp_df[column], linewidth = 2.5, color = 'skyblue', label = 'IBM')
#     ax1.plot(temp_df.index, position_temp[1], marker = '^', markersize = 10, color = 'green', label = 'BUY SIGNAL')
#     ax1.plot(temp_df.index, position_temp[2], marker = 'v', markersize = 10, color = 'r', label = 'SELL SIGNAL')
#     ax1.set_title('RSI TRADE SIGNALS')
#     ax2.plot(df['rsi'], color = 'orange', linewidth = 2.5)
#     ax2.axhline(30, linestyle = '--', linewidth = 1.5, color = 'grey')
#     ax2.axhline(70, linestyle = '--', linewidth = 1.5, color = 'grey')
#     plt.show()

profits['PnL'] = profit
profits = profits.transpose()
profit_df = pd.concat([profit_df, profits])

profit_df

# Plotting to Excel 
# profit_df.to_excel(r'/Users/halelee/Python/BH Algo/PythonResult.xlsx', sheet_name='Sheet1', index = False)




,Series 1,Series 2,Series 3,Series 4,Series 5,Series 6,Series 7,Series 8,Series 9,Series 10,Series 11,Series 12,Series 13,Series 14,Series 15,Series 16,Series 17,Series 18,Series 19,Series 20,Series 21,Series 22,Series 23,Series 24,Series 25,Series 26,Series 27,Series 28,Series 29,rsi
PnL,-656.37663,-658.976848,-51.359926,-209.406348,-174.908624,68.782374,-16.226487,-493.706052,-174.847464,-9.243686,-76.117807,-347.371767,-3537.825916,-1417.02704,-60.310364,-90.083664,-54.872112,18.385597,55.894982,91.04872,-16.480233,-1.418164,-2.9471,-24.539559,-98.762357,-50.809804,-22.476181,42.365269,25.194805,389.365561
